## Import

In [ ]:
import requests
import json
import pandas as pd
import hashlib # hash 함수용 sha256 사용할 라이브러리|
import random

## 노드 블록 정보확인

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

## transaction 입력 및 채굴(mine)

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "DuHyeon_from",
        "recipient": "DuHyeon_to",
        "amount": 100, #전달값 100
        "smart_contract": {}
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

## 노드 블록 정보 확인

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

## SMC transaction 입력

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "DuHyeon_from",
        "recipient": "smart_contract",
        "amount": 0,  #전달값은 0
        "smart_contract": {"contract_code":"print('Ｈello Ｓmart－Ｃontract')"}
}
result = requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content
json.loads(result) 

In [ ]:
contract_address = json.loads(result)['contract_address'] # contract_address 변수 지정

## 노드 블록 정보 확인

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

## 채굴(mine)

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

## 노드 블록정보 확인

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

## SMC 호출

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

In [ ]:
for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] == contract_address:
            exec( _tx['smart_contract']['contract_code']) 

## SMC 입력하기 / NFT 연동

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "DuHyeon_from",
        "recipient": "smart_contract",
        "amount": 0, #값은 0
        "smart_contract": {"contract_code":"""
myNFT = {'NFT_NAME':'DuHyeon_NFT',
         'NFT_URL': 'https://cdn.pixabay.com/photo/2022/02/18/16/09/ape-7020995_1280.png'}
                                            """}
}
result = requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content
contract_address = json.loads(result)['contract_address']

# 채굴을 통하여 거래내역을 블록에 저장한다
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

In [ ]:
contract_address # login.html에 입력

In [ ]:
# 블록 정보 호출
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

## 스마트 컨트랙트를 호출 및 실행
for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] == contract_address:
            exec( _tx['smart_contract']['contract_code'])            
myNFT

## SMC Token

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "DuHyeon_from",
        "recipient": "smart_contract",
        "amount": 0,
        "smart_contract": {
                           "contract_code" :"token_name = 'DHKR-TOKEN' \ntoken_total_volume = 100000" #10만값
                                            }
            }
result = requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content
contract_address = json.loads(result)['contract_address']

headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

In [ ]:
contract_address

In [ ]:
# 블록 정보 호출
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

## 스마트 컨트랙트를 호출 및 실행
for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] == contract_address:
            exec( _tx['smart_contract']['contract_code'])       
            break
print(token_name , "/ ", token_total_volume  )

## SMC 토큰 거래시작

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "DuHyeon_from",
        "recipient": "DuHyeon_to",
        "amount": 3,
        "smart_contract": {
                           "contract_code" :"token_name = 'DHKR-TOKEN' \ntoken_total_volume = 100000\ntoken_owner = {'token_Owner' : 10000}",
                           "contract_function_getBalance" :"""
def get_balance(user_id):
    print('{} Balance is : '.format(user_id), token_owner[user_id])
    return token_owner[user_id]
""",
    "contract_function_sendToken" :"""
def send_token(sender,recipent,amount):
    if sender in token_owner.keys():
        if get_balance(sender) > amount:
            token_owner[sender]  = token_owner[sender] - amount
            if recipent in token_owner.keys():
                token_owner[recipent]  = token_owner[recipent] + amount
            else :
                token_owner[recipent]  =  amount
            print("Transaction Completed")
            get_balance(sender) 
            get_balance(recipent) 

        else:
            return "Insufficient Balance"
    else:
        return "Unavailable Sender id"
"""
                           

                                            }
            }
result = requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content
contract_address = json.loads(result)['contract_address']


headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] == contract_address:
            exec( _tx['smart_contract']['contract_code'])
            break

In [ ]:
token_name 

In [ ]:
token_total_volume

In [ ]:
exec(_tx['smart_contract']["contract_function_getBalance"])
get_balance('token_Owner')

In [ ]:
exec(_tx['smart_contract']['contract_function_sendToken'])
send_token('token_Owner','DH-ENG',5000) # DH-ENG에게 5000값 전달

In [ ]:
exec(_tx['smart_contract']['contract_function_sendToken'])
send_token('token_Owner','DH-ENG',4900) # DH-ENG에게 4900 값 전달

In [ ]:
exec(_tx['smart_contract']['contract_function_sendToken'])
send_token('token_Owner','DH-KR', 50) # DH-KR에게 50 값 전달